# Find anomalies using ETT
In this notebook we find anomalies using the ETTs generated in previous steps. 

In [1]:
import orchest
import pandas as pd
import time
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from et import ET, ETT
from typing import Dict
import numpy as np
from utils import Anomaly

In [2]:
data = orchest.get_inputs()

In [3]:
clarify_data = data['clarify_data']
normalized_data = data['normalized_data']
# energy_consumption_dfs = clarify_data['energy_consumption']
energy_consumption_dfs = normalized_data['energy_consumption']
temperature_df = clarify_data['temperature']

et_curves_data = data['et_curves']
ett_dict: ETT = et_curves_data['ett_day_of_week']

In [4]:
building_names = list(ett_dict.keys())

anomalies_dict = {}
for i, building_name in enumerate(building_names):
    anomalies_dict[building_name] = ett_dict[building_name].get_anomalies(energy_consumption_dfs['daily'][building_name]['Totalt'], temperature_df['daily']['Voll TAN'], building_name)
    # if i < 2:
    #     print(f'{building_name}:')
    #     ett_dict[building_name].plot(energy_consumption_dfs['daily'][building_name]['Totalt'], temperature_df['daily']['Voll TAN'])


In [5]:
anomalies_from_et = {
    'anomalies_from_ett': anomalies_dict
}

In [6]:
print('outputting anomalies from et...')
orchest.output(anomalies_from_et, name='anomalies_from_et')
print('success')

outputting anomalies from et...
success
